In [ ]:
#### Notebook to munge LCD data
#### Written by: Jennifer.E.Kay@colorado.edu
#### Last update: March 11, 2022

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from metpy.units import units

from datetime import datetime

import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
import timeit
import scipy.stats as stats
import glob

In [2]:
df1 = pd.read_csv("GrandJunction_2012-2021_2891817.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
df1

df0 = pd.read_csv("GrandJunction_2011-2012_2891886.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
df0

##df0 = pd.read_csv("GrandJunction_2011-2012_2891886.csv",usecols=[1,43,44,45,49,52,55,56,57],dtype='str')

### could also try to Specify dtype option on import OR low_memory=False

,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2002-01-01T00:00:00,29,31,0.00,92,NaN,070,NaN,5
1,2002-01-01T00:09:00,28,32,NaN,87,NaN,070,NaN,5
2,2002-01-01T01:00:00,29,31,0.00,92,NaN,000,NaN,0
3,2002-01-01T01:04:00,28,30,NaN,93,NaN,230,NaN,3
4,2002-01-01T01:35:00,28,30,NaN,93,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
128687,2011-12-31T22:53:00,10,27,0.00,49,25.53,090,NaN,13
128688,2011-12-31T23:00:00,10,27,NaN,49,25.60,090,NaN,13
128689,2011-12-31T23:53:00,8,26,0.00,46,25.53,100,NaN,13
128690,2011-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
included_cols = ['DATE','HourlyDryBulbTemperature',
                 'HourlyDewPointTemperature','HourlyPrecipitation',
                'HourlyRelativeHumidity',
                'HourlyStationPressure','HourlyWindDirection',
                'HourlyWindGustSpeed','HourlyWindSpeed'] # hourly fields only
data0 = df0.loc[:, df0.columns.isin(list(included_cols))]
data1 = df1.loc[:, df1.columns.isin(list(included_cols))]
df = pd.concat([data0,data1])
print(df.shape)
df

(243495, 9)


,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2002-01-01T00:00:00,29,31,0.00,92,NaN,070,NaN,5
1,2002-01-01T00:09:00,28,32,NaN,87,NaN,070,NaN,5
2,2002-01-01T01:00:00,29,31,0.00,92,NaN,000,NaN,0
3,2002-01-01T01:04:00,28,30,NaN,93,NaN,230,NaN,3
4,2002-01-01T01:35:00,28,30,NaN,93,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
114798,2021-12-31T22:53:00,23,26,0.00,88.0,24.75,320,NaN,3
114799,2021-12-31T23:00:00,23,26,NaN,88.0,24.82,320,NaN,3
114800,2021-12-31T23:53:00,22,24,0.00,91.0,24.75,320,NaN,6
114801,2021-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
### Print the Shape of the data frame

print(df.shape)

(243495, 9)


In [5]:
#df.to_csv('72476023066_GrandJunction_2002-2021_hourlydata.csv', index=False)
### Read in your data
#df = pd.read_csv("72476023066_GrandJunction_2002-2021_hourlydata.csv",low_memory=False)
### Preview the values in the data frame by printing it
#print(df)

In [6]:
## make the datetime format is a readable datetime

##create a datetime object from a string
#foo = datetime.strptime('2012-01-01', '%Y-%m-%d')
#foo = datetime.strptime('2012-01-01T00:53:00', '%Y-%m-%dT%H:%M:%S')
#foo = datetime.strptime(df['DATE'][0], '%Y-%m-%dT%H:%M:%S')
#print(foo)

df['datetime'] = pd.to_datetime(df['DATE'], format='%Y-%m-%dT%H:%M:%S')
#df

In [7]:
## change the order of the data frame variables
df = df[['DATE','datetime','HourlyDryBulbTemperature','HourlyDewPointTemperature','HourlyPrecipitation',
                'HourlyRelativeHumidity','HourlyStationPressure','HourlyWindDirection','HourlyWindSpeed','HourlyWindGustSpeed']]
df

,DATE,datetime,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindSpeed,HourlyWindGustSpeed
0,2002-01-01T00:00:00,2002-01-01 00:00:00,31,29,0.00,92,NaN,070,5,NaN
1,2002-01-01T00:09:00,2002-01-01 00:09:00,32,28,NaN,87,NaN,070,5,NaN
2,2002-01-01T01:00:00,2002-01-01 01:00:00,31,29,0.00,92,NaN,000,0,NaN
3,2002-01-01T01:04:00,2002-01-01 01:04:00,30,28,NaN,93,NaN,230,3,NaN
4,2002-01-01T01:35:00,2002-01-01 01:35:00,30,28,NaN,93,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
114798,2021-12-31T22:53:00,2021-12-31 22:53:00,26,23,0.00,88.0,24.75,320,3,NaN
114799,2021-12-31T23:00:00,2021-12-31 23:00:00,26,23,NaN,88.0,24.82,320,3,NaN
114800,2021-12-31T23:53:00,2021-12-31 23:53:00,24,22,0.00,91.0,24.75,320,6,NaN
114801,2021-12-31T23:59:00,2021-12-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
## rename the columns, shorter and include units
df.rename(columns={'HourlyDryBulbTemperature': 'drytemp_F', 'HourlyDewPointTemperature': 'dewtemp_F',
                   'HourlyPrecipitation': 'prec_inch', 'HourlyRelativeHumidity': 'RH',
                   'HourlyStationPressure': 'pres_Hg', 'HourlyWindDirection': 'wdir',
                   'HourlyWindSpeed': 'wspd_mph', 'HourlyWindGustSpeed': 'wgust_mph'}, inplace=True)
df.iloc[0]

DATE         2002-01-01T00:00:00
datetime     2002-01-01 00:00:00
drytemp_F                     31
dewtemp_F                     29
prec_inch                   0.00
RH                            92
pres_Hg                      NaN
wdir                         070
wspd_mph                       5
wgust_mph                    NaN
Name: 0, dtype: object

In [10]:
## convert pressure from a string to a float
df["pres_Hg"] = pd.to_numeric(df["pres_Hg"], downcast="float",errors='coerce')
## Deal with the mix of text and numeric values!  Set text values to NaN using coerce!!

## convert units of pressure from inches Mercury to mbars
#print(df['pres_Hg'][1])
#print((df['pres_Hg'][1]* units.inHg).to(units.mbar))
## example converting to mbar from inches of mercury
print((25.53 * units.inHg).to(units.mbar))

df['pres_Hg']=(df.pres_Hg.values * units('inHg')).to('mbar')

df.rename(columns={'pres_Hg': 'pres_mbar'}, inplace=True)


864.5450198790573 millibar


In [14]:
df["drytemp_F"] = pd.to_numeric(df["drytemp_F"], downcast="float",errors='coerce')
df["dewtemp_F"] = pd.to_numeric(df["dewtemp_F"], downcast="float",errors='coerce')
df["prec_inch"] = pd.to_numeric(df["prec_inch"], downcast="float",errors='coerce')
df["RH"] = pd.to_numeric(df["RH"], downcast="float",errors='coerce')
df["wdir"] = pd.to_numeric(df["wdir"], downcast="float",errors='coerce')
df["wspd_mph"] = pd.to_numeric(df["wspd_mph"], downcast="float",errors='coerce')
df["wgust_mph"] = pd.to_numeric(df["wgust_mph"], downcast="float",errors='coerce')
df

,DATE,datetime,drytemp_F,dewtemp_F,prec_inch,RH,pres_mbar,wdir,wspd_mph,wgust_mph
0,2002-01-01T00:00:00,2002-01-01 00:00:00,31.0,29.0,0.0,92.0,NaN,70.0,5.0,NaN
1,2002-01-01T00:09:00,2002-01-01 00:09:00,32.0,28.0,NaN,87.0,NaN,70.0,5.0,NaN
2,2002-01-01T01:00:00,2002-01-01 01:00:00,31.0,29.0,0.0,92.0,NaN,0.0,0.0,NaN
3,2002-01-01T01:04:00,2002-01-01 01:04:00,30.0,28.0,NaN,93.0,NaN,230.0,3.0,NaN
4,2002-01-01T01:35:00,2002-01-01 01:35:00,30.0,28.0,NaN,93.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
114798,2021-12-31T22:53:00,2021-12-31 22:53:00,26.0,23.0,0.0,88.0,838.131226,320.0,3.0,NaN
114799,2021-12-31T23:00:00,2021-12-31 23:00:00,26.0,23.0,NaN,88.0,840.501709,320.0,3.0,NaN
114800,2021-12-31T23:53:00,2021-12-31 23:53:00,24.0,22.0,0.0,91.0,838.131226,320.0,6.0,NaN
114801,2021-12-31T23:59:00,2021-12-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
### deal with missing data

## copy the data array into a new array without missing values called (data_nomissing)
df_nomissing=df.copy()
df_nomissing
print(df_nomissing.shape)

option_missing='option2'

##### OPTION #1 = replace the NaN data with the mean value over the entire dataset.  Simple!
if option_missing=='option1':
    df_nomissing['drytemp_F']=df_nomissing['drytemp_F'].fillna(df['drytemp_F'].mean())
    df_nomissing['dewtemp_F']=df_nomissing['dewtemp_F'].fillna(df['dewtemp_F'].mean())
    df_nomissing['prec_inch']=df_nomissing['prec_inch'].fillna(0)
    df_nomissing['RH']=df_nomissing['RH'].fillna(df['RH'].mean())
    df_nomissing['pres_mbar']=df_nomissing['pres_mbar'].fillna(df['pres_mbar'].mean())
    df_nomissing['wdir']=df_nomissing['wdir'].fillna(df['wdir'].mean())
    df_nomissing['wspd_mph']=df_nomissing['wspd_mph'].fillna(df['wspd_mph'].mean())
    df_nomissing['wgust_mph']=df_nomissing['wgust_mph'].fillna(df['wgust_mph'].mean())
    
##### OPTION #2 = ffill: propagate last valid observation forward to next valid observation,
#### then backfill / bfill: use next valid observation to fill gap.
if option_missing=='option2':
     df_nomissing=df_nomissing.fillna(method='bfill')
     df_nomissing=df_nomissing.fillna(method='ffill')
    
df_nomissing


(243495, 10)


,DATE,datetime,drytemp_F,dewtemp_F,prec_inch,RH,pres_mbar,wdir,wspd_mph,wgust_mph
0,2002-01-01T00:00:00,2002-01-01 00:00:00,31.0,29.0,0.00,92.0,859.126892,70.0,5.0,21.0
1,2002-01-01T00:09:00,2002-01-01 00:09:00,32.0,28.0,0.00,87.0,859.126892,70.0,5.0,21.0
2,2002-01-01T01:00:00,2002-01-01 01:00:00,31.0,29.0,0.00,92.0,859.126892,0.0,0.0,21.0
3,2002-01-01T01:04:00,2002-01-01 01:04:00,30.0,28.0,0.01,93.0,859.126892,230.0,3.0,21.0
4,2002-01-01T01:35:00,2002-01-01 01:35:00,30.0,28.0,0.01,93.0,859.126892,0.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...
114798,2021-12-31T22:53:00,2021-12-31 22:53:00,26.0,23.0,0.00,88.0,838.131226,320.0,3.0,18.0
114799,2021-12-31T23:00:00,2021-12-31 23:00:00,26.0,23.0,0.00,88.0,840.501709,320.0,3.0,18.0
114800,2021-12-31T23:53:00,2021-12-31 23:53:00,24.0,22.0,0.00,91.0,838.131226,320.0,6.0,18.0
114801,2021-12-31T23:59:00,2021-12-31 23:59:00,24.0,22.0,0.00,91.0,838.131226,320.0,6.0,18.0


In [37]:
df_nomissing.to_csv('72476023066_GrandJunction_2002-2021_hourlydata_munged.csv', index=False)

In [ ]:
#df.to_csv('goo.csv', index=False)